# Digital Music Concert Archive (DMCA) Data Anlysis

From the period of ~2005 to now, the concert archives have been recorded in a pseudo-database within FileMaker Pro. Unfortunately, since this is not a standard database, we can't use robust tools within python that handle SQL (at least not without some conversion).  

For now, though, we can get the tables into data frames for `pandas` analysis. Since one or two of the tables are incomplete, we can start with the "contributors" table. Instead of what you would expect, where one performer gets one unique (primary) key, a new key was produced for _each_ performer, for _each_ piece.  

We could handle this in a couple different ways, but since the main table that we are interested in, the "pieces" table, has keys that correspond to the "performers" table via the key "Piece_ID", we could try to normalize on that, producing a table that has all the Piece IDs, with each row then having a list of performers.   

In [3]:
import pandas as pd

Let's turn the csv into a dataframe:

In [4]:
%%time
contrib_data = 'ca_contrib-cleanup.csv'
df = pd.read_csv(contrib_data, index_col='Contrib_ID')

CPU times: user 24 ms, sys: 8 ms, total: 32 ms
Wall time: 49.7 ms


Let's look at a slice of the table:

In [6]:
df[:9]

,Piece_ID,Instrument,Name
Contrib_ID,,,
1,2009102501,Cello,"Erenyi, Erica"
2,2009102501,Piano,"Lee, Yvonne"
3,2009102501,Violin,"Négyesy, János"
4,2009102501,"Violin, Viola","Nykter, Päivikki"
5,2009102501,Counter Tenor,"Toledo, John"
6,2009102502,Cello,"Erenyi, Erica"
7,2009102502,Piano,"Lee, Yvonne"
8,2009102502,Violin,"Négyesy, János"
9,2009102502,"Violin, Viola","Nykter, Päivikki"


## Fundamental Database Issues

Note how for the performer "Negyesy, Janos", he has two unique Contributor IDs. It turns out he has many more in this table. We need to get around that, so as mentioned above, let's just use Piece ID to get a list of performers for each piece. We can do this via a self-join or pivot.  

However, if we try to pivot on Piece ID, `pandas` will complain the index contains duplicates. To get around this, we need to make an index on Piece ID that groups them into unique groups:

In [9]:
df['idx'] = df.groupby('Piece_ID').cumcount()

Now we have that unique index of our piece IDs, we can merge on those to just get the names for each piece:

In [12]:
df_norm = df.pivot(index='Piece_ID', columns='idx')[['Name']]
df_norm[:19]

Name                                \
idx                                 0                             1    
Piece_ID                                                               
2007002                 Ohnishi, Takae                          None   
2007003                 Ohnishi, Takae                          None   
2007004                 Ohnishi, Takae                          None   
2007005                 Ohnishi, Takae                          None   
2007006                 Ohnishi, Takae                          None   
2007007                 Ohnishi, Takae                          None   
2007008   La Jolla Symphony and Chorus                          None   
2007009                  Sutter, Wendy  La Jolla Symphony and Chorus   
2007010   La Jolla Symphony and Chorus                          None   
2007011              Zelickman, Robert                          None   
2007012              Zelickman, Robert                          None   
2007013              Zelickman, Robert                          None   
2007014              Zelickman, Robert               Oliveira, Fabio   
2007015                Gattegno, Eliot                 Wubbels, Eric   
2007016                Gattegno, Eliot                 Wubbels, Eric   
2007017                Gattegno, Eliot                 Wubbels, Eric   
2007018                Gattegno, Eliot                 Wubbels, Eric   
2007019                Gattegno, Eliot                 Wubbels, Eric   
2007020                Gattegno, Eliot                 Wubbels, Eric   

                                                                            \
idx                  2                  3             4     5     6     7    
Piece_ID                                                                     
2007002            None               None          None  None  None  None   
2007003            None               None          None  None  None  None   
2007004            None               None          None  None  None  None   
2007005            None               None          None  None  None  None   
2007006            None               None          None  None  None  None   
2007007            None               None          None  None  None  None   
2007008            None               None          None  None  None  None   
2007009            None               None          None  None  None  None   
2007010            None               None          None  None  None  None   
2007011            None               None          None  None  None  None   
2007012            None               None          None  None  None  None   
2007013            None               None          None  None  None  None   
2007014   Dresser, Mark  McAllister, Colin  Menezes, Flo  None  None  None   
2007015            None               None          None  None  None  None   
2007016            None               None          None  None  None  None   
2007017            None               None          None  None  None  None   
2007018            None               None          None  None  None  None   
2007019            None               None          None  None  None  None   
2007020            None               None          None  None  None  None   

                      ...                                                   \
idx         8     9   ...     37    38    39    40    41    42    43    44   
Piece_ID              ...                                                    
2007002   None  None  ...   None  None  None  None  None  None  None  None   
2007003   None  None  ...   None  None  None  None  None  None  None  None   
2007004   None  None  ...   None  None  None  None  None  None  None  None   
2007005   None  None  ...   None  None  None  None  None  None  None  None   
2007006   None  None  ...   None  None  None  None  None  None  None  None   
2007007   None  None  ...   None  None  None  None  None  None  None  None   
2007008   None  None  ...   No

So now that we concatenated vertically, we see there's an interesting issue... some of these pieces have a whole lot of performers. In fact, from the above output we now know there's a piece with 47 performers!